<a href="https://colab.research.google.com/github/a2kiti/colab_web_demo/blob/master/CGAN_fashionMnist.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
from keras.datasets import mnist, fashion_mnist
from keras.layers import Input, Dense, Reshape, Flatten, Dropout, Conv2D, Conv2DTranspose
from keras.layers import BatchNormalization, Activation, UpSampling2D, concatenate
from keras.layers.advanced_activations import LeakyReLU
from keras.models import Model, load_model

from keras.optimizers import Adam

import sys

import numpy as np

from scipy.misc import imread, imsave
import os
import requests, json
import io

In [0]:
img_rows = 28 
img_cols = 28
channels = 1
class_num = 10

# Dimensions of latent variable
z_dim = 100

In [0]:
# discriminator model
input_img = Input((img_rows, img_cols, channels + class_num))
x = Conv2D(64, 4, strides=2, padding='same')(input_img)
x = LeakyReLU(0.2)(x)
x = Dropout(0.25)(x)

x = Conv2D(128, 4, strides=2, padding='same')(x)
x = LeakyReLU(0.2)(x)
x = Dropout(0.25)(x)

x = Conv2D(256, 4, strides=2, padding='same')(x)
x = BatchNormalization()(x)
x = LeakyReLU(0.2)(x)
x = Dropout(0.25)(x)

x = Flatten()(x)

x = Dense(1)(x)
x = Activation('sigmoid')(x)

dis = Model(input_img, x, name="discriminator")

dis.summary()

dis_optimizer = Adam(lr=1e-5, beta_1=0.1)
dis.compile(loss='binary_crossentropy', optimizer=dis_optimizer, metrics=['accuracy'])
dis.trainable = False

In [0]:
# Generator model
noise_shape = Input((z_dim + class_num,))

x = Dense(128*7*7)(noise_shape)
x = BatchNormalization()(x)
x = LeakyReLU(0.2)(x)

x = Reshape((7,7,128))(x)

x = UpSampling2D((2,2))(x)
x = Conv2DTranspose(filters=64, kernel_size=4, strides=1, padding='same')(x)
x = BatchNormalization()(x)
x = LeakyReLU(0.2)(x)

x = UpSampling2D((2,2))(x)
x = Conv2DTranspose(filters=32, kernel_size=4, strides=1, padding='same')(x)
x = BatchNormalization()(x)
x = LeakyReLU(0.2)(x)

x = Conv2DTranspose(filters=channels, kernel_size=4, strides=1, padding='same')(x)
x = Activation('tanh')(x)

gen = Model(noise_shape, x, name="generator")

gen.summary()


In [0]:
z = Input((z_dim,))
label = Input((class_num,))
label_img = Input((img_rows,img_cols,class_num,))

z_with_label = concatenate([z, label], axis=-1)

img = gen(z_with_label)
img_with_label = concatenate([img, label_img], axis=3)

valid = dis(img_with_label)
com = Model([z, label, label_img], valid, name="combined")
com.summary()

com_optimizer = Adam(lr=.8e-4, beta_1=0.5)
com.compile(loss='binary_crossentropy', optimizer=com_optimizer)

In [0]:
# Loading mnist
(X_train, y_train), (_, _) = fashion_mnist.load_data()
# (X_train, y_train), (_, _) = mnist.load_data()

# Normalization
X_train = (X_train.astype(np.float32) - 127.5) / 127.5
X_train = np.expand_dims(X_train, axis=3)

In [0]:
#Connecting to googledrive
from google.colab import drive
drive.mount('/content/gdrive')

In [0]:
modelpath = 'gdrive/My Drive/Colab Notebooks/CGAN_model.hdf5'

In [0]:
epochs = 3000
batch_size = 128
save_interval=1000

half_batch = int(batch_size / 2)

for epoch in range(epochs):

    # ---------------------
    #  Training of Discriminator
    # ---------------------

    # Generating images from Generator
    noise = np.random.normal(0, 1, (half_batch, z_dim))
    noise_label_index = np.random.randint(0, class_num, half_batch)
    noise_label = np.zeros((half_batch, class_num))
    for i in range(half_batch):
        noise_label[i, noise_label_index[i]] = 1
    noise_with_label = np.concatenate((noise, noise_label),axis=1)

    gen_imgs = gen.predict(noise_with_label)
    label_imgs = np.zeros((half_batch, img_rows, img_cols, class_num))
    for i in range(half_batch):
        label_imgs[i, :, :, noise_label_index[i]] = 1
    gen_imgs_with_label = np.concatenate((gen_imgs, label_imgs),axis=3)

    # Picking from train data
    idx = np.random.randint(0, X_train.shape[0], half_batch)
    imgs = X_train[idx]
    label_imgs = np.zeros((half_batch, img_rows, img_cols, class_num))
    for i in range(half_batch):
        label_imgs[i, :, :, y_train[idx][i]] = 1
    imgs_with_label = np.concatenate((imgs, label_imgs),axis=3)

    
    real_label = np.ones((half_batch, 1))
    fake_label = np.zeros((half_batch, 1))
    
    d_loss_real = dis.train_on_batch(imgs_with_label, real_label)
    d_loss_fake = dis.train_on_batch(gen_imgs_with_label, fake_label)
    d_loss = 0.5 * np.add(d_loss_real, d_loss_fake)
    
    # ---------------------
    #  Training of Generator
    # ---------------------

    noise = np.random.normal(0, 1, (batch_size, z_dim))
    noise_label = np.zeros((batch_size, class_num))
    noise_label_imgs = np.zeros((batch_size, img_rows, img_cols, class_num))
    for i in range(batch_size):
        label_index = np.random.randint(0,class_num)
        noise_label[i, label_index] = 1
        noise_label_imgs[i, :, :, label_index] = 1
       
    
    valid_y = np.ones((batch_size, 1))

    # Train the generator
    g_loss = com.train_on_batch([noise, noise_label, noise_label_imgs], valid_y)


    if epoch % 100 == 0:
        print ("%d [D loss: %f, acc.: %.2f%%] [G loss: %f]" % (epoch, d_loss[0], 100*d_loss[1], g_loss))
        


In [0]:
gen.save(modelpath)